In [ ]:
# Model architecture inspired by Z. Hameed and B. Garcia-Zapirain, "Sentiment classification using a single-layered BiLSTM model", IEEE Access, vol. 8, pp. 73992-74001, 2020.
#!pip install fasttext

import numpy as np
import fasttext
import fasttext.util
    
# Download the cc.en.300.bin English model beforehand from FastText website
model_ft = fasttext.load_model('LOCAL_PATH_TO_cc.en.300.bin') 

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import re

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Bidirectional, LSTM, Input, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate

from keras.layers.embeddings import Embedding

def process_text(document):
     
    # Remove extra white space from text
    document = re.sub(r'\s+', ' ', document, flags=re.I)
         
    # Remove all the special characters from text
    document = re.sub(r'\W', ' ', str(document))
 
    return document

In [ ]:
df = pd.read_csv('LOCAL_PATH_TO_DATASET')
df = df[['Emotion','Statement']]
display(df.head())

,Emotion,Statement
0,joy,"Thank you , Steven . I accept your advice ."
1,surprise,"Oh my God, I can't believe I have two-two chil..."
2,neutral,"Look, it's not that big of a deal, so Monica a..."
3,anger,And I wanna know why?!!
4,joy,"And, ah, you know, your fooling around with her."


In [ ]:
from tqdm import tqdm

nltk.download('wordnet')

df['preprocessedStatement'] = df.Statement.apply(process_text)
display(df.head())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Emotion,Statement,preprocessedStatement
0,joy,"Thank you , Steven . I accept your advice .",Thank you Steven I accept your advice
1,surprise,"Oh my God, I can't believe I have two-two chil...",Oh my God I can t believe I have two two chil...
2,neutral,"Look, it's not that big of a deal, so Monica a...",Look it s not that big of a deal so Monica a...
3,anger,And I wanna know why?!!,And I wanna know why
4,joy,"And, ah, you know, your fooling around with her.",And ah you know your fooling around with her


In [ ]:
max_length = df.preprocessedStatement.apply(lambda x: len(x.split())).max()

t = Tokenizer()
t.fit_on_texts(df['preprocessedStatement'] )
vocab_size = len(t.word_index) + 1
encoded_tweets = t.texts_to_sequences(df['preprocessedStatement'] )
X = pad_sequences(encoded_tweets, maxlen=max_length, padding='post')

In [ ]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = model_ft.get_word_vector(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
# Encode labels in column 'Emotion'. 
df['Emotion'] = le.fit_transform(df['Emotion']) 
y = df.pop('Emotion')
y_new = tf.keras.utils.to_categorical(y, num_classes=7)
print(y_new)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [ ]:
list(le.inverse_transform([0,1,2,3,4,5,6]))

['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

In [ ]:
y.value_counts()

3    15193
4     6436
6     3459
0     2629
5     2152
1      714
2      532
Name: Emotion, dtype: int64

In [ ]:
# 2 options to handle imbalanced dataset: class_weight or focal loss
class_weight = {0: 6, 1: 22, 2: 30, 3: 1, 4: 2, 5: 7, 6: 5}

In [ ]:
from tensorflow.keras import activations

def focal_loss(gamma=2., alpha=4., from_logits=False):

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax if from_logits is False.
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.cast(y_true, dtype=tf.float32)
        y_pred = tf.cast(y_pred, dtype=tf.float32)
        if from_logits:
            y_pred = activations.softmax(y_pred)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y_new, test_size=0.05, stratify=y_new)

x_val = x_train[:100]
y_val = y_train[:100]
x_train = x_train[100:]
y_train = y_train[100:]

In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=3)
input_layer = Input(shape=(max_length), )
x = Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=False)(input_layer)
x = Bidirectional(LSTM(32, return_sequences=True))(x)
x_a = GlobalMaxPool1D()(x)
x_b = GlobalAveragePooling1D()(x)
x = concatenate([x_a,x_b])
x = Dense(64, activation="relu")(x)
x = Dense(7, activation='softmax')(x)
model_ft = Model(inputs=input_layer, outputs=x)
model_ft.compile(loss=focal_loss(alpha=1), optimizer='adam', metrics=['accuracy']) #use if you are using focal loss
#model_ft.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #use if you are using class_weight
model_ft.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 231)]        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 231, 300)     3194400     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 231, 64)      85248       embedding_1[0][0]                
__________________________________________________________________________________________________
global_max_pooling1d_1 (GlobalM (None, 64)           0           bidirectional_1[0][0]            
____________________________________________________________________________________________

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model_ft,show_shapes= True)

In [ ]:
model_ft.fit(x_train, y_train, epochs = 20, validation_data=(x_val, y_val), callbacks=[callback]) #insert class_weight=class_weight if using class_weight

Epoch 1/20
921/921 [==============================] - 89s 81ms/step - loss: 0.7916 - accuracy: 0.5540 - val_loss: 0.8233 - val_accuracy: 0.5500
Epoch 2/20
921/921 [==============================] - 74s 80ms/step - loss: 0.6654 - accuracy: 0.6171 - val_loss: 0.7998 - val_accuracy: 0.5600
Epoch 3/20
921/921 [==============================] - 74s 81ms/step - loss: 0.6212 - accuracy: 0.6375 - val_loss: 0.7716 - val_accuracy: 0.5800
Epoch 4/20
921/921 [==============================] - 74s 80ms/step - loss: 0.5900 - accuracy: 0.6520 - val_loss: 0.7609 - val_accuracy: 0.5900
Epoch 5/20
921/921 [==============================] - 74s 80ms/step - loss: 0.5635 - accuracy: 0.6658 - val_loss: 0.7605 - val_accuracy: 0.6000
Epoch 6/20
921/921 [==============================] - 74s 80ms/step - loss: 0.5411 - accuracy: 0.6734 - val_loss: 0.7044 - val_accuracy: 0.6300
Epoch 7/20
921/921 [==============================] - 74s 81ms/step - loss: 0.5194 - accuracy: 0.6826 - val_loss: 0.7249 - val_accuracy:

In [ ]:
y_pred = model_ft.predict(x_test)

In [ ]:
y_pred = np.argmax(y_pred, 1)
y_test = np.argmax(y_test, 1)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.40      0.29      0.33       131
           1       0.33      0.14      0.20        36
           2       0.50      0.19      0.28        26
           3       0.81      0.84      0.83       760
           4       0.52      0.65      0.58       322
           5       0.54      0.45      0.49       108
           6       0.58      0.53      0.56       173

    accuracy                           0.67      1556
   macro avg       0.53      0.44      0.47      1556
weighted avg       0.66      0.67      0.66      1556

